In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np
tqdm.pandas()
import time

from google.cloud import bigquery
import gcsfs

client = bigquery.Client(location="US")
print("Client creating using default project: {}".format(client.project))

Client creating using default project: res-nbcupea-dev-ds-sandbox-001


# See what the trained node2vec titles are like

In [3]:
token_vocab_list="gs://edc-dev/kubeflowpipelines-default/tfx_pipeline_output/node2vec_sports_syn_0_1_0/Transform/transform_graph/18561/transform_fn/assets/node_vocab_txt"
fs = gcsfs.GCSFileSystem(project="res-nbcupea-dev-ds-sandbox-001")
with fs.open(token_vocab_list, "r") as fid:
    tags_list = fid.read().split("\n")
tags_list = [tag for tag in tags_list if tag != ""]
#tags_list_id = [tag for tag in tags_list if len(tag)==36] # inSeason Series Id
#tags_list_id
tags_list

['News',
 'TV',
 'Sports',
 'Talk',
 'covid-19',
 'new',
 'Entertainment',
 'Football',
 'Business & Finance',
 'coronavirus',
 'reports',
 '022ae9a1-d2ac-3238-b686-96c2a5ce26ba',
 'president',
 'news',
 'discuss',
 'trump',
 'joins',
 'today',
 'SportingEvent',
 'Comedy',
 'first',
 'mike',
 'show',
 'Subgenre:Talk',
 'Reality',
 'one',
 'us',
 'c04236ee-ca40-3a75-a008-230eaa805ba4',
 '39337ec8-062e-32ba-afc3-541adf683fce',
 'pandemic',
 'chris',
 'says',
 'nbc',
 'joe',
 'former',
 'cnbcs',
 'florio',
 '235b584f-ef32-38fe-8c51-e769f320257f',
 'day',
 'nbcs',
 'biden',
 '2020',
 'ddca41be-7eff-3441-96df-14b01bb41629',
 'world',
 'people',
 'watch',
 'talks',
 'get',
 'back',
 '2c1b4ec5-a425-32ed-a230-26638587fbec',
 'Soccer',
 'house',
 'ed68bf9f-dd4f-34b0-805f-142f3483c997',
 'season',
 'time',
 'look',
 'two',
 '38cfc6e9-47ba-33e0-a857-143193c2b7d6',
 'Drama',
 'c909dfb4-1532-3d0c-a532-0a5b3798efec',
 '700e1a8a-dcdd-3bea-a86b-7a03a595f26a',
 'a63d70f5-4640-348c-acc2-7b5c02ea5a65',
 

In [31]:
query = """
SELECT DISTINCT TitleDetails_title, COALESCE(InSeasonSeries_Id, TitleId) as InSeasonSeries_Id
FROM res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.ContentMetadataView
WHERE (InSeasonSeries_Id IS NOT NULL OR TitleId IS NOT NULL)
AND TitleDetails_title IS NOT NULL
"""
df_id = client.query(query=query, location="US").to_dataframe()
df_id["Is_in_node2vec"] = df_id["InSeasonSeries_Id"].isin(tags_list_id)
df_id

,TitleDetails_title,InSeasonSeries_Id,Is_in_node2vec
0,"4 Downs with Rich: Eisen talks NFC East, NO, MIN",185ccb94-d814-3841-bdc7-57918269b3d6,True
1,New warning about robocall scams acting to be ...,b7c145e4-7741-33ae-ab40-6830f82627f3,True
2,Medical workers on new COVID-19 surge,b002e00b-37a2-328c-b30e-84b0600dc51e,True
3,"Trump demands schools reopen, no plan for doin...",1b4fdcee-647c-3c4e-86b9-a0a66e2c2123,True
4,Cramer: This is not your typical 'Santa Claus ...,5803f697-8ae9-3bb6-9553-690257635e17,True
...,...,...,...
92489,Superthotics - Relieve Pain with Every Step,66aae2a7-cc8d-32ec-80d8-a817c48ca52a,True
92490,Premier League Soccer,699bff2c-062b-3df5-a421-90df97b109de,True
92491,Driven,498d57e9-7f9d-39d2-9793-44c650b774e3,True
92492,NBA Basketball,30f0f0e8-ae01-326e-a2cc-56c2db975bae,True


In [28]:
df_id.loc[df["TitleDetails_title"].str.contains("Mr. Mer")]

,TitleDetails_title,InSeasonSeries_Id,Is_in_node2vec
32288,Mr. Mercedes,5d416922-f6d1-379b-88e7-6d92f3b94630,True


In [29]:
df_id.loc[~df["Is_in_node2vec"]]

,TitleDetails_title,InSeasonSeries_Id,Is_in_node2vec


# See what the test frame is like

In [15]:
TITLES_QUERY = """
    SELECT
        TitleDetails_title, 
        TitleType, 
        STRING_AGG(DISTINCT TitleDetails_longsynopsis, ' ') as TitleDetails_longsynopsis, 
        cid.content_ordinal_id,
        ARRAY_AGG(DISTINCT COALESCE(InSeasonSeries_Id, TitleId)) as InSeasonSeries_Id
    FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.ContentMetadataView` cmv
    LEFT JOIN `res-nbcupea-dev-ds-sandbox-001.recsystem.ContentOrdinalId` cid
        ON LOWER(cmv.TitleDetails_title) = LOWER(cid.program_title)
    WHERE 
        TitleDetails_longsynopsis IS NOT NULL
        AND cid.content_ordinal_id IS NOT NULL
    GROUP BY 
        TitleDetails_title, 
        TitleType, 
        cid.content_ordinal_id
"""
#.drop_duplicates("TitleDetails_title")\
df_test = client.query(query=TITLES_QUERY, location="US").to_dataframe()\
    .reset_index(drop=True)
#df_test["title_id"] = df_test["InSeasonSeries_Id"].apply(lambda x: tags_list.index(x))
#df_test = df_test.groupby(by=["TitleDetails_title", "content_ordinal_id"], as_index=False).agg(
#    {"TitleType": "first", "TitleDetails_longsynopsis": "first", "InSeasonSeries_Id": list, "title_id":list}
#)
df_test

,TitleDetails_title,TitleType,TitleDetails_longsynopsis,content_ordinal_id,InSeasonSeries_Id
0,Man on the Moon,Movie,Milos Forman's film spotlights Andy Kaufman's ...,538,[e4fdef3c-42c9-3b98-a0f1-dcc511eee1c7]
1,Age of Dinosaurs,Movie,A former firefighter fights to rescue his teen...,52,"[64f8eb79-af08-39d1-b036-03c2195a6631, 72e5a23..."
2,The Big Lebowski,Movie,"Bowling buddies (Jeff Bridges, John Goodman) b...",1153,[80941756-f81e-3bb3-a28f-1254d7c610de]
3,The Front Page,Movie,The managing editor (Walter Matthau) of a 1920...,849,[e2e4a2fb-e419-38f8-8bd7-82568da584cd]
4,Middle Men,Movie,A Texas businessman (Luke Wilson) tries unsucc...,558,[2d9d15e5-8625-340e-9f52-6af7e1db2011]
...,...,...,...,...,...
2315,Down in the Valley,Programme,A lawman (David Morse) is suspicious of a char...,297,[1820f73e-cdab-3b17-8e6e-23f5c71dfe6e]
2316,Action Cats,Programme,Action Cat toys are plastic armor and weapons ...,1317,[3a2b2416-db91-333c-8135-aaaca9478b27]
2317,Fear,Programme,Deranged suitor (Mark Wahlberg) torments teen ...,342,[4c241203-1b75-3712-9d8a-473cf4f99979]
2318,Thank You Notes: Apple Watch & more,Programme,"Jimmy pens thank you notes to smirks, Presiden...",1344,[0d1e9dfb-a282-3d86-b506-229c67ddd320]


In [11]:
df_test.loc[df_test["TitleDetails_title"].str.contains("WWE")]

,TitleDetails_title,content_ordinal_id,TitleType,TitleDetails_longsynopsis,InSeasonSeries_Id,title_id
1739,The Best of WWE,1209,Programme,The best matches in history of the World Wrest...,"[4706c0cb-dfda-3868-b90f-20a853a38829, ba2d138...","[76901, 436, 76578, 76933, 27568, 25520, 76591]"
2178,WWE Friday Night SmackDown,2292,Programme,"The 2019 WWE Draft kicks off; Seth Rollins, re...",[dc7e37c0-64ac-384d-a2d5-5ac0c51ecece],[543]
2179,WWE Monday Night RAW,2301,Programme,The Undertaker vs. Isaac Yankem. Ultimate Warr...,[2c1b4ec5-a425-32ed-a230-26638587fbec],[49]
2180,WWE NXT,2341,Programme,No Way Jose vs. Killian Dain. The NXT Champion...,[6f9f5c75-c4ed-3567-a850-52f21e30dc5c],[381]
2181,WWE Ruthless Aggression,1216,Programme,With Vince McMahon's declaration of Ruthless A...,"[25e85f2a-efc6-3e69-b3a6-780fa93e5f55, 94b2617...","[2537, 29148, 19708, 72206, 23191, 23452]"
2182,WWE The Bump,2363,Programme,Goldberg previews his WWE Championship match a...,[db5f77b4-04f5-3d52-be39-c32d9b0ead86],[1614]
2183,WWE Untold,1212,Programme,"At WrestleMania 21, two of the greatest techni...","[7755243f-c486-3954-b66e-3f29c765df3f, 8552ae8...","[29444, 71479, 20432, 21455, 19529, 24131, 601..."


In [12]:
out = model_title(np.array([29444, 71479, 20432])).numpy()
out = out / np.sqrt(np.sum(out**2, axis=1, keepdims=True))
out @ out.T

array([[0.99999994, 0.37976286, 0.18052791],
       [0.37976286, 1.0000001 , 0.17118567],
       [0.18052791, 0.17118567, 0.99999994]], dtype=float32)

# Try to run the eval metrics with titles

In [7]:
import tensorflow as tf
import tensorflow_text

In [8]:
TITLES_QUERY = """
    SELECT
        TitleDetails_title, 
        TitleType, 
        STRING_AGG(DISTINCT TitleDetails_longsynopsis, ' ') as TitleDetails_longsynopsis, 
        cid.content_ordinal_id,
        ARRAY_AGG(DISTINCT COALESCE(InSeasonSeries_Id, TitleId) as InSeasonSeries_Id)
    FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.ContentMetadataView` cmv
    LEFT JOIN `res-nbcupea-dev-ds-sandbox-001.recsystem.ContentOrdinalId` cid
        ON LOWER(cmv.TitleDetails_title) = LOWER(cid.program_title)
    WHERE 
        TitleDetails_longsynopsis IS NOT NULL
        AND cid.content_ordinal_id IS NOT NULL
    GROUP BY 
        TitleDetails_title, 
        TitleType, 
        cid.content_ordinal_id
"""
date_start = "2021-2-01"
date_end = "2021-4-01"

PREV_WINDOW = 10
TEST_WINDOW = 5
DATA_LENGTH = PREV_WINDOW + TEST_WINDOW
NUM_SAMPLES = 50000

USERS_QUERY = ("""
    SELECT psv2.user_ordinal_id, 
        session_date, 
        session_timestamp, 
        ss.content_id
    FROM `res-nbcupea-dev-ds-sandbox-001.recsystem.PlaySequenceV2` as psv2,
    UNNEST(session_sequence) AS ss
    LEFT JOIN (
    SELECT user_ordinal_id, 
        COUNT(DISTINCT(ss.content_id)),
        row_number() over (order by farm_fingerprint(concat(user_ordinal_id, '3')) ) as seqnum
    FROM `res-nbcupea-dev-ds-sandbox-001.recsystem.PlaySequenceV2`, 
    UNNEST(session_sequence) AS ss
    WHERE session_date >= "{0}"
        AND session_date < "{1}"
    GROUP BY user_ordinal_id
    HAVING COUNT(DISTINCT(ss.content_id)) > {2}
    ) filtered_data
    ON filtered_data.user_ordinal_id = psv2.user_ordinal_id
    WHERE seqnum <= {3}
    ORDER BY session_timestamp
    """).format(date_start, date_end, DATA_LENGTH, NUM_SAMPLES)

def ntail(g, n):
    return g._selected_obj[g.cumcount() >= n]

In [9]:
print("Loading model")
model_path = "gs://metadata-bucket-base/tfx-metadata-dev-pipeline-output/metadata_dev_edc_base_0_0_5/Trainer/model/23309/serving_model_dir"
model = tf.saved_model.load(model_path)
embed_path = "gs://edc-dev/kubeflowpipelines-default/tfx_pipeline_output/node2vec_sports_syn_0_1_1/Trainer/model/19130/serving_model_dir"
model_title = tf.keras.models.load_model(embed_path).get_layer("Embedding")

        
### Load User Data
print("Loading data")
client = bigquery.Client()
raw_user_data = client.query(USERS_QUERY).result().to_dataframe()


### Create embeddings
unscored_titles = client.query(TITLES_QUERY) \
                        .result() \
                        .to_dataframe() \
                        .drop_duplicates(subset=['TitleDetails_title']) \
                        .reset_index()       
unscored_titles["title_id"] = unscored_titles["InSeasonSeries_Id"].apply(lambda x: tags_list.index(x))
unscored_titles = unscored_titles.groupby(by=["TitleDetails_title", "content_ordinal_id"], as_index=False).agg(
    {"TitleType": "first", "TitleDetails_longsynopsis": "first", "InSeasonSeries_Id": list, "title_id":list}
)

Loading model


/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/layers/core.py:1045: UserWarning: models.node2vec.node2vec is not loaded, but a Lambda layer uses it. It may cause errors.
  , UserWarning)


Loading data


In [10]:
print("Start making predictions on synopsis")
tnow = time.time()
res = []

input_data = {"synopsis": unscored_titles['TitleDetails_longsynopsis'].values[:, None], 
              "title": tf.ragged.constant(df_test["title_id"].values)
    #"tokens": tf.ragged.constant(unscored_titles["tokens"].values).to_sparse(),
    #"kewords": tf.ragged.constant(unscored_titles["keywords"].values).to_sparse(),
    }
dataset = tf.data.Dataset.from_tensor_slices(input_data).batch(50)

for batch in tqdm(dataset):
    y = model({"synopsis":batch["synopsis"]}) # predict synopsis
    t = model_title(batch["title"]) # predict title embeddings
    t = tf.math.reduce_mean(t, axis=1)
    y = tf.concat([y, t], axis=1)
    res.append(y)

used_time = time.time() - tnow
print(f"Successfully made predictions on synopsis: {used_time:.2f} s")

######## TODO: Look at this block ####################
f = tf.concat(res, axis=0).numpy()
# del res
# del dataset
# del model
preds = pd.DataFrame(f)
preds['pred'] = preds.iloc[:,:].values.tolist()
preds['pred'] = preds['pred'].apply(np.asarray)
preds['content_ordinal_id'] = unscored_titles['content_ordinal_id']
preds = preds[['pred','content_ordinal_id']]

user_data = pd.merge(raw_user_data,
                     preds,
                     how="inner", 
                     left_on="content_id", 
                     right_on="content_ordinal_id") \
                     .drop(columns=['content_ordinal_id'])

test_window_data = user_data.sort_values(['user_ordinal_id','session_timestamp']) \
                                .groupby(['user_ordinal_id'],sort=True)

history_df = test_window_data.head(PREV_WINDOW).reset_index()

# get all entries after the first 20
after_prev = ntail(test_window_data, PREV_WINDOW).reset_index().drop_duplicates(['user_ordinal_id', 'content_id'])

#remove content that was in the history_df
after_prev = pd.merge(after_prev,
                      history_df[['user_ordinal_id', 'content_id', 'index']],
                      how="left",
                      on=['user_ordinal_id', 'content_id'])

# remove entries where the show was already watched in history
after_prev = after_prev.loc[after_prev['index_y'].isnull()] 

test_df = after_prev.sort_values(['user_ordinal_id', 'session_timestamp']) \
                    .groupby(['user_ordinal_id'],sort=True) \
                    .head(TEST_WINDOW) \
                    .drop(columns=['index_y','index_x']) \
                    .reset_index()

avg_emb = history_df.groupby(['user_ordinal_id'])['pred'] \
                    .apply(np.sum)

tnow = time.time()
avg_emb = avg_emb.apply(lambda x: np.asarray([i / PREV_WINDOW for i in x]))
used_time = time.time() - tnow
print(f"Average embed: {used_time:.2f} s")

##############################################################

tnow = time.time()
#cos_sim = avg_emb.apply(lambda x: [1 - spatial.distance.cosine(u, x) for u in preds['pred']])
avg_emb_mat = np.stack(avg_emb.values)
preds_emb_mat = np.stack(preds["pred"].values)
# L2 Normalize
avg_emb_mat = avg_emb_mat / np.sqrt(np.sum(avg_emb_mat**2, axis=1, keepdims=True))
preds_emb_mat = preds_emb_mat / np.sqrt(np.sum(preds_emb_mat**2, axis=1, keepdims=True))
# Cosine similarity Scoring
user_content_mat = avg_emb_mat @ preds_emb_mat.T
del avg_emb_mat
del preds_emb_mat
print("cosine similarity")
print("mean:", np.nanmean(user_content_mat.ravel()))
print("std:", np.nanstd(user_content_mat.ravel()))
print("median:", np.nanmedian(user_content_mat.ravel()))
user_content_mat = np.nan_to_num(user_content_mat, nan=-1)
cos_sim = pd.Series(list(user_content_mat), index=avg_emb.index, name="user_ordinal_id")
used_time = time.time() - tnow
print(f"Cosine sim: {used_time:.2f}s")

# make cosine sim for things in the history -1 so they don't get predicted
tnow = time.time()
df_joined_history = history_df[["user_ordinal_id", "content_id"]].merge(
    preds.reset_index(drop=False)[["index", "content_ordinal_id"]], 
    left_on="content_id", right_on="content_ordinal_id")
df_user_hist = df_joined_history.groupby(by=["user_ordinal_id"]).agg({"index":list})
df_user_hist = df_user_hist.join(cos_sim.to_frame(name="cos_sim"), on="user_ordinal_id")

def zeroing_func(pdf):
    pdf["cos_sim"][pdf["index"]] = -1
    return pdf

df_user_hist = df_user_hist.apply(zeroing_func, axis=1)
# Reset cos_sim
cos_sim = df_user_hist["cos_sim"]
print(f"cos sim slicer for loop: {used_time} s")

## Predict / Eval on test-data using cos sim
# prev_window: how many prior shows to average into user embedding
# test_window: how many shows to allow into the future for correctly guessing.
#              i.e test_window=1 means predict next show exactly,
#                  test_window=5 means prediction must be within next 5 shows watched

recall = {}
precision = {}
coverage = {}
seen = {}
accuracy = {}
total = len(user_data['user_ordinal_id'].unique())

tnow = time.time()
top_all = cos_sim.apply(lambda x: np.argsort(x))
for n in [-1,-5,-10]:
    print(f"top {n}")
    top_with_ids = np.take(preds["content_ordinal_id"].values, np.stack(top_all.values)[:, n:])
    top_with_ids = pd.Series(list(top_with_ids), index=cos_sim.index, name="user_ordinal_id")\
        .to_frame(name="content_ordinal_id").reset_index()
    print("Computing metrics for each user")
    for _ , userid in top_with_ids.iterrows():
        future_data = set(test_df.loc[test_df['user_ordinal_id'] == userid['user_ordinal_id']].content_id.values.tolist())
        if len(future_data) < 1:
            continue
        topn = set(userid["content_ordinal_id"])

        if n in coverage.keys():
            coverage[n] = coverage[n].union(topn)
            seen[n] = seen[n].union(future_data)
            recall[n].append(len(topn.intersection(future_data)) / len(future_data))
            precision[n].append(len(topn.intersection(future_data)) / -n)
        else:
            coverage[n] = topn
            seen[n] = future_data
            recall[n] = [len(topn.intersection(future_data))]
            precision[n] = [len(topn.intersection(future_data)) / -n]
        if len(topn.intersection(future_data)) >= 1:
            if n not in accuracy.keys():
                accuracy[n] = 1
            else:
                accuracy[n] += 1
used_time = time.time() - tnow
print(f"Double for loop to get top 1/5/10 metrics: {used_time} s")

metric_vals = {}
metric_vals['Precision@1'] = sum(precision[-1]) / total
metric_vals['Precision@5'] = sum(precision[-5]) / total
metric_vals['Precision@10'] = sum(precision[-10]) / total

metric_vals['Recall@1'] = sum(recall[-1]) / total
metric_vals['Recall@5'] = sum(recall[-5]) / total
metric_vals['Recall@10'] = sum(recall[-10]) / total

metric_vals['Accuracy@1'] = accuracy[-1] / total
metric_vals['Accuracy@5'] = accuracy[-5] / total
metric_vals['Accuracy@10'] = accuracy[-10] / total

metric_vals['Coverage@1'] = len(coverage[-1]) / len(seen[-1])
metric_vals['Coverage@5'] = len(coverage[-5]) / len(seen[-5])
metric_vals['Coverage@10'] = len(coverage[-10]) / len(seen[-10])

# Print the results
for k, v in metric_vals.items():
    print(k, ":", v)

  0%|          | 0/46 [00:00<?, ?it/s]

Start making predictions on synopsis


100%|██████████| 46/46 [00:12<00:00,  3.58it/s]


Successfully made predictions on synopsis: 12.87 s
Average embed: 4.02 s
cosine similarity
mean: 0.5678980878278682
std: 0.18357908655636224
median: 0.6007389928245559
Cosine sim: 5.07s
cos sim slicer for loop: 5.06972861289978 s
top -1
Computing metrics for each user
top -5
Computing metrics for each user
top -10
Computing metrics for each user
Double for loop to get top 1/5/10 metrics: 145.77238512039185 s
Precision@1 : 0.01736
Precision@5 : 0.016736000000000827
Precision@10 : 0.013338000000001252
Recall@1 : 0.003471999999999974
Recall@5 : 0.016736000000000827
Recall@10 : 0.026676000000002503
Accuracy@1 : 0.01736
Accuracy@5 : 0.07516
Accuracy@10 : 0.11548
Coverage@1 : 0.5302593659942363
Coverage@5 : 0.7853025936599424
Coverage@10 : 0.8919308357348703


# Prepare the data for training

In [4]:
query = """
    WITH title_ids AS (
        SELECT
            TitleDetails_title, 
            ARRAY_AGG(DISTINCT COALESCE(InSeasonSeries_Id, TitleId)) as InSeasonSeries_Id
        FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.ContentMetadataView` cmv
        WHERE 
            TitleDetails_longsynopsis IS NOT NULL
        GROUP BY 
            TitleDetails_title, 
            TitleType
    )
    
    SELECT a.program_title, a.program_type, a.program_longsynopsis, a.tags, 
    CASE 
        WHEN ARRAY_LENGTH(b.InSeasonSeries_Id) IS NULL THEN ["movie"]
        ELSE b.InSeasonSeries_Id
    END AS InSeasonSeries_Id,
    COALESCE(ARRAY_LENGTH(b.InSeasonSeries_ID), 0) AS titles_length
    FROM `res-nbcupea-dev-ds-sandbox-001.metadata_enhancement.synopsis_dylan_150tag_with_tokens_and_keywords` a
    LEFT JOIN title_ids b
    ON LOWER(a.program_title) = LOWER(b.TitleDetails_title)
"""
table_id = f"{client.project}.metadata_enhancement.synopsis_titles_dylan_150tag"
job_config = bigquery.QueryJobConfig(destination=table_id, write_disposition="WRITE_TRUNCATE")

df_train = client.query(query=query, job_config=job_config, location="US").to_dataframe()
df_train

,program_title,program_type,program_longsynopsis,tags,InSeasonSeries_Id,titles_length
0,When Sharks Attack: Deep Dives,Episode,Cancun never has a shark attack on its shore u...,"[Nature, Documentary, Animals, eng, Episode, E...",[movie],0
1,Primetime: Mind Games,Episode,A woman and her 8-year-old daughter tell about...,"[spa, Documentary, Episode, eng]",[movie],0
2,"Yours Truly, Miss Chinatown",Movie,The stories of three young women vying for the...,"[Movie, Documentary, eng]",[movie],0
3,River Journey IMAX,Movie,A mythical downstream odyssey takes viewers th...,"[eng, Movie, Documentary]",[movie],0
4,Les herbes folles,Movie,Marguerite no contaba con que le robaran el bo...,"[spa, Drama, Movie]",[movie],0
...,...,...,...,...,...,...
514622,Genetic Roulette: The Gamble of Our Lives,Movie,El cineasta Jeffrey M. Smith explora los poten...,"[Documentary, Movie, spa]",[movie],0
514623,The Removalists,Movie,Trouble brews when a veteran police officer an...,"[eng, Drama, Movie]",[movie],0
514624,2018 ETV Debates,Episode,Runoff debate between Governor Henry McMaster ...,"[Local, Episode, Politics & Government, eng]",[movie],0
514625,El Nombre de Las Flores,Movie,Margarita apenas conoció a sus padres y ahora ...,"[Movie, spa, Drama]",[movie],0


In [48]:
tmp = df_train.loc[df_train["titles_length"] > 0]
tmp

,program_title,program_type,program_longsynopsis,tags,InSeasonSeries_Id,titles_length
70,Meatballs,Movie,A zany summer camp counselor leads his misfit ...,"[Teens, Comedy, teens (ages 13-14), eng, Movie...",[3af763ab-7e97-3c58-9884-5a900c9041ef],1
146,Supporting Characters,Movie,Dos editores de cine de Nueva York intentan ma...,"[spa, Comedy, Movie]",[60fe6d3f-9b32-3df8-b4da-f6b5cd827bef],1
170,Jumanji,Movie,Alan Parris queda atrapado durante 25 años en ...,"[Fantasy, spa, Action & Adventure, Movie, Chil...",[a8100f18-8f28-3c47-ae33-0e4a62903b6b],1
183,CB4,Movie,"Tres jóvenes (Chris Rock, Allen Payne, Deezer ...","[Movie, spa, Comedy]",[73685271-dd45-3d91-a4ee-ba4715ac3cb4],1
223,Valerian and the City of a Thousand Planets,Movie,"Two special operatives (Dane DeHaan, Cara Dele...","[Movie, Action & Adventure, Science fiction, eng]",[e6376bcb-95a1-332a-acff-31892b2a10bd],1
...,...,...,...,...,...,...
514371,Street League Skateboarding,Episode,Eight women compete for the first time ever at...,"[eng, spa, Sports non-event, Episode, Sports]",[040dccea-2101-3e2b-9169-0d81059fbf58],1
514393,Crazy,Movie,Guitarist Hank Garland (Waylon Payne) emerges ...,"[eng, Drama, Movie, Biography, Musical]",[6d7b94a0-4065-39f8-b7f6-c886afac1f92],1
514522,My Bloody Valentine,Movie,Un inexperto minero de carbón causa un acciden...,"[Movie, Thriller, spa, Horror, Holiday]",[7d43f11a-dd43-3740-856c-72a8608fc15c],1
514593,Hannibal,Movie,Hace 10 años el doctor Lecter escapó de las au...,"[Movie, Thriller, spa, Drama]",[358c106f-fc01-3666-a1d5-a705ecbf6fcc],1


In [44]:
tmp.loc[tmp["program_title"].str.contains("Yello")]

,program_title,program_type,program_longsynopsis,tags,InSeasonSeries_Id
229847,Yellowstone,Movie,El cineasta Kieth Merrill muestra la belleza d...,"[Documentary, spa, Movie]",[51aab181-4250-3334-8c75-fe55da453fe9]
265916,The Yellow Mountain,Movie,Andy Martín llega a un pueblo del oeste para r...,"[Western, spa, Movie]",[2bd7e641-86d9-3ef0-ac23-767f089f7f5c]
272258,The Yellow Mountain,Movie,"Two partners (Lex Barker, Howard Duff) fight o...","[eng, Movie, Western]",[2bd7e641-86d9-3ef0-ac23-767f089f7f5c]
290919,Yellowstone,Movie,Crooks die looking for their loot around hot g...,"[Crime, eng, Movie, Drama, Mystery]",[51aab181-4250-3334-8c75-fe55da453fe9]
406344,Yellowstone,Movie,Soar over majestic Yellowstone National Park a...,"[Movie, eng, Documentary]",[51aab181-4250-3334-8c75-fe55da453fe9]
444221,Yellowstone,Episode,El Parque Natural de Yellowstone se encuentra ...,"[Episode, older teens (ages 15+), Drama, Natur...",[51aab181-4250-3334-8c75-fe55da453fe9]


In [50]:
tmp.drop_duplicates(subset=["program_title"])

,program_title,program_type,program_longsynopsis,tags,InSeasonSeries_Id,titles_length
70,Meatballs,Movie,A zany summer camp counselor leads his misfit ...,"[Teens, Comedy, teens (ages 13-14), eng, Movie...",[3af763ab-7e97-3c58-9884-5a900c9041ef],1
146,Supporting Characters,Movie,Dos editores de cine de Nueva York intentan ma...,"[spa, Comedy, Movie]",[60fe6d3f-9b32-3df8-b4da-f6b5cd827bef],1
170,Jumanji,Movie,Alan Parris queda atrapado durante 25 años en ...,"[Fantasy, spa, Action & Adventure, Movie, Chil...",[a8100f18-8f28-3c47-ae33-0e4a62903b6b],1
183,CB4,Movie,"Tres jóvenes (Chris Rock, Allen Payne, Deezer ...","[Movie, spa, Comedy]",[73685271-dd45-3d91-a4ee-ba4715ac3cb4],1
223,Valerian and the City of a Thousand Planets,Movie,"Two special operatives (Dane DeHaan, Cara Dele...","[Movie, Action & Adventure, Science fiction, eng]",[e6376bcb-95a1-332a-acff-31892b2a10bd],1
...,...,...,...,...,...,...
513614,The Deposition,Movie,"Adam reunites with his former lover, Jill, at ...","[Mystery, Drama, Crime drama, eng, Thriller, M...",[316283a4-6e44-382d-8c91-fd998860ba11],1
513949,Jeopardy!,Episode,Wayne Brady; Vivica A. Fox; Ashton Kutcher. IB...,"[eng, Game Show, spa, kids (ages 5-9), Episode]","[c7041121-226d-35f4-aa63-85cd51e301c7, 7e6625e...",4
514096,Saying Goodbye,Episode,An elderly widower turns to work to hide his g...,"[Health, Drama, Episode, eng, Local]",[85ee22b0-1229-3cec-ab84-cf0b2637da75],1
514155,Odd Mom Out,Episode,"Por el día, Jill lanza la nueva línea de Brook...","[spa, Sitcom, Comedy, Episode, eng, teens (age...",[e9f9003e-66f6-33c0-b99a-6c0dd6257d95],1
